**PROYECTO NLP DPIR 2023**

In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.tokens import Doc, Span
from spacy.matcher import PhraseMatcher
import json

In [2]:
archivo = pd.read_excel("BD version 1 NLP.xlsx", sheet_name= "Hoja2")
archivo.head(5)


,Renglon,Nombre del Producto,Caracteristicas Solicitadas,Presentacion,Unidad de Medida,Precio de Referencia,Rubro,Nog,Evento,Categoria
0,1,CERA PARA HUESO (PALMITATO ISOPROPÍLICO),CERA PARA HUESO: A BASE DE CERA DE ABEJA (30 ...,NaN,GRAMO,21.500000,MATERIAL HEMOSTATICO,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
1,2,CERA PARA HUESO (JALEA DE PETROLATO),CERA PARA HUESO: A BASE DE CERA DE ABEJA (20 ...,NaN,GRAMO,7.960967,MATERIAL HEMOSTATICO,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
2,3,"SONDA LEVIN, No. 8 FR, DE PVC GRADO MÉDICO","SONDA LEVIN NASOGÁSTRICA, No. 8 FR, DE 40 A 42...",NaN,UNIDAD,3.055372,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
3,4,"SONDA LEVIN, No. 12 FR, DE PVC GRADO MÉDICO","SONDA LEVIN NASOGÁSTRICA, No.12 FR, DE 48 A 50...",NaN,UNIDAD,6.541582,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
4,5,"SONDA LEVIN, No. 10 FR, DE PVC GRADO MÉDICO","SONDA LEVIN NASOGÁSTRICA, No. 10 FR, DE 40 A ...",NaN,UNIDAD,3.900875,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico


In [3]:
# Crear un nuevo data frame sin la columna caraceristicas y presentacion, prueba 1

BD = archivo.drop(columns=["Caracteristicas Solicitadas", "Presentacion","Nog"])
BD.head(5)

,Renglon,Nombre del Producto,Unidad de Medida,Precio de Referencia,Rubro,Evento,Categoria
0,1,CERA PARA HUESO (PALMITATO ISOPROPÍLICO),GRAMO,21.500000,MATERIAL HEMOSTATICO,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
1,2,CERA PARA HUESO (JALEA DE PETROLATO),GRAMO,7.960967,MATERIAL HEMOSTATICO,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
2,3,"SONDA LEVIN, No. 8 FR, DE PVC GRADO MÉDICO",UNIDAD,3.055372,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
3,4,"SONDA LEVIN, No. 12 FR, DE PVC GRADO MÉDICO",UNIDAD,6.541582,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
4,5,"SONDA LEVIN, No. 10 FR, DE PVC GRADO MÉDICO",UNIDAD,3.900875,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico


In [4]:
!python -m spacy download es_core_news_md

2023-10-31 21:30:42.192338: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 21:30:42.192410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 21:30:42.192449: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 21:30:43.571911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 20.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [42]:
etiqueta_ner

'Medicamentos'

In [6]:
from spacy.training.example import Example

# Crear una lista de tuplas con nombres de productos y etiquetas NER
productos_ner = [(nombre, categoria) for nombre, categoria in zip(BD['Nombre del Producto'], BD['Categoria'])]

# Mapea las categorías a las etiquetas NER
categoria_a_etiqueta = {
    "Material medico": "Material medico",
    "Medicamentos": "Medicamentos",
    "Alimentacion": "Alimentacion",
    "Plaguicidas": "Plaguicidas"
}

# Cargar el modelo preentrenado
nlp = spacy.load("es_core_news_md")

# Comprueba si el componente NER ya existe en el pipeline, si no, créalo
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe("ner")

# Agregar tus etiquetas NER personalizadas sin eliminar las etiquetas originales
ner_labels = ["Material medico", "Medicamentos", "Alimentacion", "Plaguicidas"]
for label in ner_labels:
    ner.add_label(label)

# Lista para almacenar los datos de entrenamiento etiquetados
training_data = []

for nombre, categoria in productos_ner:
    # Limpiar la categoría eliminando espacios en blanco al principio y al final
    categoria = categoria.strip()
    # Mapea la categoría a las etiquetas NER correspondiente
    etiqueta_ner = categoria_a_etiqueta.get(categoria, "Otro")

    # Crear un diccionario de anotación
    annotation = {
        "start": 0,
        "end": len(nombre),
        "label": etiqueta_ner
    }

    # Crear un diccionario de entrenamiento para el texto y las anotaciones
    training_example = {
        "text": nombre,
        "entities": [annotation]
    }

    # Agregar el ejemplo de entrenamiento a la lista
    training_data.append(training_example)

# Guardar los datos de entrenamiento en un archivo JSONL
with open("training_data.jsonl", "w") as jsonl_file:
    for item in training_data:
        jsonl_file.write(json.dumps(item) + "\n")

# Datos de entrenamiento etiquetados
train_data = training_data

# Procesar los datos de entrenamiento y realizar la actualización
for example in train_data:
    text = example['text']
    entities = example['entities']
    entities_tuples = [(entity['start'], entity['end'], entity['label']) for entity in entities]
    example = Example.from_dict(nlp.make_doc(text), {"entities": entities_tuples})

    # Actualizar el modelo
    nlp.update([example], drop=0.5)

# Imprimir las etiquetas NER personalizadas en el modelo
print("Etiquetas NER personalizadas en el modelo nlp:", ner.labels)

Etiquetas NER personalizadas en el modelo nlp: ('Alimentacion', 'LOC', 'MISC', 'Material medico', 'Medicamentos', 'ORG', 'PER', 'Plaguicidas')


In [7]:
# Guardar el modelo actualizado
nlp.to_disk("modelo_ner_personalizado")

In [41]:
# Cargar el modelo entrenado
nlp_updated = spacy.load("modelo_ner_personalizado")

# Probar el modelo actualizado
text = "DESAYUNOS\n (Alumnos, Instructores, Capacitaciones, Ascensos, Especialidades, Talleres y Eventos) Semana 1,  Día 7"
doc = nlp_updated(text)
for ent in doc.ents:
    print("Texto: ",ent.text,"\n""Entidad:", ent.label_)

Texto:  DESAYUNOS
 (Alumnos, Instructores, Capacitaciones, Ascensos, Especialidades, Talleres y Eventos) Semana 1,  Día 7 
Entidad: Medicamentos
